In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoProcessor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [3]:
df_allCarabids = pd.read_csv("/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/AllCarabids.csv", sep="\t")
df_insect1M = df_allCarabids[df_allCarabids["filepath"].str.startswith("Insect-1M")].copy()
df_insect1M.rename(columns={"filepath": "ImageFilePath", "class": "ScientificName"}, inplace=True)
cols = ['ImageFilePath', 'ScientificName']
df = df_insect1M[cols]
df['ImageFilePath'] = "/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/" + df['ImageFilePath']
df = df.fillna("Unknown")
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata
1,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "facebook/levit-128S"

model = AutoModel.from_pretrained(model_name).eval().to(device)

processor = AutoProcessor.from_pretrained(model_name)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [5]:
def extract_features(image_path) :
    
    image = Image.open(image_path).convert("RGB")
    
    inputs = processor(images=image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    
    features = outputs.last_hidden_state.mean(dim=1)
    
    return features.cpu().numpy().flatten()
    

In [6]:
# Separate known and unknown samples
df_known = df[df["ScientificName"] != "Unknown"]
df_unknown = df[df["ScientificName"] == "Unknown"]

# Extract features for known samples (only passing img)
X_known = np.vstack([extract_features(img) for img in tqdm(df_known["ImageFilePath"], desc="Processing ...")])

# Encode labels only for known species
le = LabelEncoder()
y_known = le.fit_transform(df_known["ScientificName"])

df_indices_known = df_known.index

# Train-test split only on known species
X_train, X_test_known, y_train, y_test_known, train_idx, test_idx_known = train_test_split(
    X_known, y_known, df_indices_known, test_size=0.2, random_state=42
)

# Extract features for unknown samples (only passing img)
X_test_unknown = np.vstack([extract_features(img) for img in tqdm(df_unknown["ImageFilePath"], desc="Processing Unknown ...")])
test_idx_unknown = df_unknown.index

# Assign a placeholder label (-1) for unknown samples
y_test_unknown = np.full(len(X_test_unknown), -1)

# Combine known and unknown test sets
X_test = np.vstack([X_test_known, X_test_unknown])
y_test = np.concatenate([y_test_known, y_test_unknown])
test_idx = np.concatenate([test_idx_known, test_idx_unknown])

print(f"Train set: {X_train.shape}, Test set: {X_test.shape}")


Processing ...:   0%|          | 0/20278 [00:00<?, ?it/s]

Processing Unknown ...:   0%|          | 0/4328 [00:00<?, ?it/s]

Train set: (16222, 384), Test set: (8384, 384)


In [7]:
seed = 99

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=11),  
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in tqdm(models.items(), desc="Training Models", leave=True):
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


Training Models:   0%|          | 0/4 [00:00<?, ?it/s]

NaiveBayes                | Acc: 8.06% | Prec: 5.60% | Rec: 8.06% | F1: 5.30% | Bal Acc: 9.43% | MCC: 0.0932
LogisticRegression        | Acc: 15.99% | Prec: 9.73% | Rec: 15.99% | F1: 10.97% | Bal Acc: 18.38% | MCC: 0.1855
NearestNeighbor           | Acc: 10.00% | Prec: 6.86% | Rec: 10.00% | F1: 6.95% | Bal Acc: 9.41% | MCC: 0.1154
MLP_Baseline              | Acc: 14.01% | Prec: 9.14% | Rec: 14.01% | F1: 10.29% | Bal Acc: 16.05% | MCC: 0.1625


In [8]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
8222,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela willistoni,Habroscelimorpha dorsalis,Cicindela repanda,Cicindela tranquebarica,Cicindelidia schauppii
15309,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela hirticollis,Amara angustata,Cicindela hirticollis,Cicindela hirticollis,Cicindela hirticollis


In [9]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.08063,0.055961,0.08063,0.05304,0.094349,0.093247
LogisticRegression,LogisticRegression,0.159948,0.097273,0.159948,0.109717,0.183787,0.185483
NearestNeighbor,NearestNeighbor,0.099952,0.068637,0.099952,0.069522,0.094087,0.115381
MLP_Baseline,MLP_Baseline,0.140148,0.0914,0.140148,0.102905,0.160514,0.162468


In [10]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Insect1M/3.LeViT-Insect1M-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Insect1M/3.LeViT-Insect1M-species-metrics.csv", index=False)